# Installation


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/sberbank-ai/Real-ESRGAN.git to /tmp/pip-req-build-dhhtf9c2
  Running command git clone -q https://github.com/sberbank-ai/Real-ESRGAN.git /tmp/pip-req-build-dhhtf9c2
     |████████████████████████████████| 163 kB 2.1 MB/s 
  Created wheel for RealESRGAN: filename=RealESRGAN-1.0-py3-none-any.whl size=9135 sha256=71d9475d9be9b8d309fa1b52b472fd0e53c1273a078e9b3518ef201dd994d505
  Stored in directory: /tmp/pip-ephem-wheel-cache-w8wnvmqt/wheels/70/03/3f/182d6997f8fb3be93d5a86397612242994e94c63c8e31fe5b5
Successfully built RealESRGAN


# Upscale images

In [ ]:
#@title Create model
from RealESRGAN import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "2" #@param ["2", "4", "8"] {allow-input: false}

model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')

device: cuda


/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:591: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Weights downloaded to: weights/RealESRGAN_x2.pth


## OPTION: move files and fix filenames

In [ ]:
!cp -r /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/*.png /content/drive/MyDrive/AI/realesrgan_upscaling/input/

In [ ]:
%cd /content/drive/MyDrive/AI/realesrgan_upscaling/input/

/content/drive/MyDrive/AI/realesrgan_upscaling/input


In [ ]:
%%sh
for file in *-*.png
 do
   newName="${file#*-}"
   mv -- "${file}" "${newName}"
 done

## upscale

In [ ]:
from glob import glob
input_path = "/content/drive/MyDrive/AI/realesrgan_upscaling/input"
filename_list = sorted(glob(input_path+"/*", recursive = True))

In [ ]:
#@title Upload and upscale images or .tar archives
import os
from google.colab import files
import shutil
from io import BytesIO
import io
import tarfile

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = io.BufferedReader(buff)
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(buff.getvalue())
    return img_tar_info, fp

def process_tar(path_to_tar):
    processing_tar = tarfile.open(path_to_tar, mode='r')
    result_tar_path = os.path.join('results/', os.path.basename(path_to_tar))
    save_tar = tarfile.open(result_tar_path, 'w')

    for c, member in enumerate(processing_tar):
        print(f'{c}, processing {member.name}')

        if not member.name.endswith(IMAGE_FORMATS):
            continue

        try:
            img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
            img_lr = Image.open(img_bytes, mode='r').convert('RGB')
        except Exception as err:
            print(f'Unable to open file {member.name}, skipping')
            continue

        img_sr = model.predict(np.array(img_lr))
        # adding to save_tar
        img_tar_info, fp = image_to_tar_format(img_sr, member.name)
        save_tar.addfile(img_tar_info, fp)

    processing_tar.close()
    save_tar.close()    
    print(f'Finished! Archive saved to {result_tar_path}')

def process_input(filename):
    if tarfile.is_tarfile(filename):
        process_tar(filename)
    else:
        result_image_path = os.path.join('results/', os.path.basename(filename))
        image = Image.open(filename).convert('RGB')
        sr_image = model.predict(np.array(image))
        sr_image.save(result_image_path)
        print(f'Finished! Image saved to {result_image_path}')

# upload files
# uploaded = files.upload()
for filename in filename_list:
    print('Processing:', filename)
    process_input(filename)

# # upload files
# uploaded = files.upload()
# for filename in uploaded.keys():
#     print('Processing:', filename)
#     process_input(filename)

Processing: /content/drive/MyDrive/AI/realesrgan_upscaling/input/img0001image.png
Finished! Image saved to results/img0001image.png
Processing: /content/drive/MyDrive/AI/realesrgan_upscaling/input/img0002image.png
Finished! Image saved to results/img0002image.png
Processing: /content/drive/MyDrive/AI/realesrgan_upscaling/input/img0003image.png
Finished! Image saved to results/img0003image.png
Processing: /content/drive/MyDrive/AI/realesrgan_upscaling/input/img0004image.png
Finished! Image saved to results/img0004image.png
Processing: /content/drive/MyDrive/AI/realesrgan_upscaling/input/img0005image.png
Finished! Image saved to results/img0005image.png
Processing: /content/drive/MyDrive/AI/realesrgan_upscaling/input/img0006image.png
Finished! Image saved to results/img0006image.png
Processing: /content/drive/MyDrive/AI/realesrgan_upscaling/input/img0007image.png
Finished! Image saved to results/img0007image.png
Processing: /content/drive/MyDrive/AI/realesrgan_upscaling/input/img0008imag